In [1]:
# Problem tanımı: Verilen kelimelerden anlamlı türkçe cümlelerin oluşturulması

In [2]:
# import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
# Veri seti: chatgpt ile oluşturulan 100 adet günlük hayat cümlesi
data = [
"Bugün hava çok güzel, dışarı çıkalım mı?",
"Kahvaltıda ne yedin?",
"İşten ne zaman döneceksin?",
"Marketten ekmek almayı unutma.",
"Bu akşam ne yemek yapalım?",
"Telefonum şarjı bitmiş, şarj aletini verebilir misin?",
"Yarın sabah erken kalkmam lazım.",
"Film izlemek ister misin?",
"Başım ağrıyor, ilaç içtim.",
"Arabayı yıkatmam gerekiyor.",
"Çocuklar ödevlerini yaptı mı?",
"Hafta sonu ne yapalım?",
"Su içmeyi unutma, susuz kalmışsın.",
"Yeni ayakkabılarını beğendim.",
"Toplantı saat kaçta başlıyor?",
"Çantamı nerede bıraktım acaba?",
"Bugün çok yoruldum.",
"Kediyi besledin mi?",
"Faturaları ödedim mi diye kontrol et.",
"Parka gidelim mi?",
"Bu elbise sana çok yakışmış.",
"Kahve içmek istiyorum.",
"Trafik çok yoğun bugün.",
"Kitabı bitirdin mi?",
"Anahtarları masanın üstüne koydum.",
"Hava soğuk, montunu giy.",
"Alışveriş listesi yapalım.",
"Televizyonu kapatır mısın?",
"Yarın doktora randevum var.",
"Çiçekleri sulamayı unutma.",
"Ne kadar güzel bir gün!",
"Uyumadan önce dişlerini fırçala.",
"Bulaşıkları yıkadım.",
"Müzik açalım mı?",
"Yemek çok lezzetli olmuş.",
"Telefon çalıyor, açar mısın?",
"Pencereyi açabilir misin, hava alalım.",
"Çay demledim, içelim mi?",
"Evde süt kalmamış.",
"Yatmadan önce kitap okuyorum.",
"Sabah spor yaptım.",
"Arkadaşım bugün aradı.",
"Çöpü atmayı unutmuşum.",
"Bu şarkıyı çok seviyorum.",
"Dışarıda yağmur yağıyor.",
"Geç kaldım, acele etmem lazım.",
"Hediyeni çok beğendim, teşekkürler.",
"Odanı toplar mısın?",
"İyi geceler, tatlı rüyalar.",
"Günaydın, nasılsın?",
"Akşam yemeği için dışarı mı çıksak?",
"Bu dizinin yeni bölümü çıkmış mı?",
"Çamaşırları asmayı unutma.",
"Hava durumu ne diyor yarın için?",
"Ofise geç kalıyorum, trafik felaket.",
"Su faturası ne kadar gelmiş?",
"Yeni bir tarif denemek istiyorum.",
"Köpek gezdirilmeyi bekliyor.",
"Pazardan meyve aldım, çok taze.",
"Uyku tutmadı, biraz kitap okuyayım.",
"Arkadaşlarla buluşalım mı bu akşam?",
"Buzdolabını temizlemem lazım.",
"Sinema bileti aldım, geliyor musun?",
"Dişçiye randevu almayı unuttum.",
"Bu tişörtü nereden almıştın?",
"Yoruldum, biraz dinleneyim.",
"Çay bitti, marketten alalım.",
"Komşu yine müzik açmış yüksek sesle.",
"Yarın erken toplantım var.",
"Saçımı kestirmem gerekiyor.",
"Evde yoğurt kalmamış.",
"Bisikletle gezmeye çıkalım mı?",
"Bu filmi daha önce izlemiştik.",
"Kredi kartı ekstresini kontrol ettim.",
"Bahçedeki otları yolmam lazım.",
"Kışlık kıyafetleri çıkarayım mı?",
"Telefonuma yeni kılıf aldım.",
"Yemekten sonra tatlı yiyelim mi?",
"Uçak bileti fiyatlarına baktın mı?",
"Odanı havalandır biraz.",
"Yeni bir kahve makinesi bakıyorum.",
"Ders çalışmaya başla lütfen.",
"Market indirimdeymiş, gidelim mi?",
"Bu şarkıyı indirir misin?",
"Geçen hafta ne kadar güzel geçmişti.",
"Banyoyu temizledim bugün.",
"Yastık kılıflarını değiştirelim.",
"Haftaya tatil planı yapalım.",
"Gözlüğüm nerede kaldı acaba?",
"Sabah koşuya çıktım.",
"Misafir gelecek, evi toparlayalım.",
"Bu kremi denedim, çok iyi.",
"İnternet yavaşladı yine.",
"Yarın doğum günü partisi var.",
"Çantama kalem koydum mu?",
"Akşam yürüyüşe çıkalım mı?",
"Kitaplığımı düzenlemem lazım.",
"Yeni bir hobi edinmek istiyorum.",
"Dondurma yiyelim mi?",
"İyi akşamlar, hoşça kal."
]

In [4]:
# -- Preprocessing --
# tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index) + 1 # +1: padding için
print(total_words)

280


In [5]:
input_sequences = []
for text in data:
  token_list = tokenizer.texts_to_sequences([text])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

print(input_sequences)

[[7, 10], [7, 10, 2], [7, 10, 2, 16], [7, 10, 2, 16, 26], [7, 10, 2, 16, 26, 17], [7, 10, 2, 16, 26, 17, 1], [58, 4], [58, 4, 59], [60, 4], [60, 4, 61], [60, 4, 61, 62], [27, 63], [27, 63, 28], [27, 63, 28, 11], [3, 12], [3, 12, 4], [3, 12, 4, 29], [3, 12, 4, 29, 13], [64, 65], [64, 65, 66], [64, 65, 66, 67], [64, 65, 66, 67, 68], [64, 65, 66, 67, 68, 69], [64, 65, 66, 67, 68, 69, 14], [8, 18], [8, 18, 30], [8, 18, 30, 70], [8, 18, 30, 70, 9], [71, 72], [71, 72, 73], [71, 72, 73, 14], [74, 75], [74, 75, 76], [74, 75, 76, 77], [78, 79], [78, 79, 31], [80, 81], [80, 81, 82], [80, 81, 82, 1], [32, 83], [32, 83, 4], [32, 83, 4, 13], [33, 84], [33, 84, 11], [33, 84, 11, 85], [33, 84, 11, 85, 86], [6, 87], [6, 87, 34], [88, 89], [88, 89, 90], [88, 89, 90, 91], [92, 35], [92, 35, 93], [92, 35, 93, 36], [7, 2], [7, 2, 37], [94, 95], [94, 95, 5], [96, 97], [96, 97, 5], [96, 97, 5, 98], [96, 97, 5, 98, 38], [96, 97, 5, 98, 38, 99], [100, 39], [100, 39, 5], [3, 101], [3, 101, 102], [3, 101, 102, 

In [6]:
"""
[[7, 10], [7, 10, 2], [7, 10, 2, 16], [7, 10, 2, 16, 26], [7, 10, 2, 16, 26, 17], [7, 10, 2, 16, 26, 17, 1]

Bugün (7) hava(10) çok(2) güzel(16), dışarı(26) çıkalım(17) mı(1)?
"""

'\n[[7, 10], [7, 10, 2], [7, 10, 2, 16], [7, 10, 2, 16, 26], [7, 10, 2, 16, 26, 17], [7, 10, 2, 16, 26, 17, 1]\n\nBugün (7) hava(10) çok(2) güzel(16), dışarı(26) çıkalım(17) mı(1)?\n'

In [7]:
# padding: farklı uzunluktaki dizileri sabitle
max_sequence_lenght = max(len(x) for x in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen = max_sequence_lenght, padding='pre')
print(input_sequences)

[[  0   0   0 ...   0   7  10]
 [  0   0   0 ...   7  10   2]
 [  0   0   0 ...  10   2  16]
 ...
 [  0   0   0 ...   0  52 277]
 [  0   0   0 ...  52 277 278]
 [  0   0   0 ... 277 278 279]]


In [8]:
# X, y ayrımı
X = input_sequences[:, :-1] # n-1 kelimeyi giriş olarak seç
y = input_sequences[:, -1]  # n inci kelimeyi tahmin et

In [9]:
# hedef değişkene one hot encoding =
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [10]:
# -- LSTM Training --
model = Sequential()
model.add(Embedding(total_words, 50, input_length=X.shape[1])) # embedding katmanı
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [11]:
# eğitimi başlat
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0065 - loss: 5.6347
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0383 - loss: 5.6204
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0615 - loss: 5.5910
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0381 - loss: 5.4560
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0400 - loss: 5.2555
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0379 - loss: 5.2660
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0190 - loss: 5.2437
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0289 - loss: 5.1792
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0325 - loss: 5.2104
Epoch 10/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0592 - loss: 5.1802
Epoch 11/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0475 - loss: 5.1601
Epoch 12/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy

In [12]:
# örnek üretim testi (metin üretimi)
def generate_text(seed_text, next_words):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0] # tokenization
    token_list = pad_sequences([token_list], maxlen=max_sequence_lenght-1, padding='pre') # padding
    predicted_probs = model.predict(token_list, verbose=0)
    predicted_index = np.argmax(predicted_probs, axis=-1)[0]
    predicted_word = tokenizer.index_word[predicted_index]
    seed_text = seed_text +" "+predicted_word
  return seed_text

In [20]:
print(generate_text("Bu", 5))

Bu şarkıyı çok seviyorum yakışmış aletini
